<a href="https://colab.research.google.com/github/SON-OF-EDDY/pytorch-tutorial/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# convert mnist image files into tensor of 4 dims
# num images, height, width, color channel

transform = transforms.ToTensor()

In [4]:
# train data
train_data = datasets.MNIST(root='/cnn_data',train=True,download=True,transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 105959934.05it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 22358009.20it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 23968072.32it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15119467.28it/s]


Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
# test data
test_data = datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [20]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [21]:
# small batch of 10
train_loader = DataLoader(train_data,batch_size=10,shuffle=True)
test_loader = DataLoader(test_data,batch_size=10,shuffle=False)

In [22]:
# define our CNN model
# describe convolutional layer
# 2 convo layers
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)



In [23]:
# grab one mnist image
for i, (X_Train,y_train) in enumerate(train_data):
  break

In [24]:
x = X_Train.view(1,1,28,28)

In [25]:
# perform convolution
x = F.relu(conv1(x))


In [26]:
x

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.3009, 0.3009, 0.3009,  ..., 0.3009, 0.3009, 0.3009],
          [0.3009, 0.3009, 0.3009,  ..., 0.3009, 0.3009, 0.3009],
          [0.3009, 0.3009, 0.3009,  ..., 0.3009, 0.3009, 0.3009],
          ...,
          [0.3009, 0.3009, 0.1783,  ..., 0.3009, 0.3009, 0.3009],
          [0.3009, 0.3009, 0.2474,  ..., 0.3009, 0.3009, 0.3009],
          [0.3009, 0.3009, 0.3009,  ..., 0.3009, 0.3009, 0.3009]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [27]:
x.shape

torch.Size([1, 6, 26, 26])

In [28]:
# pass through pooling layer
x = F.max_pool2d(x,2,2) # kernel and stride of 2

In [29]:
# second convo layer
x = F.relu(conv2(x))

In [30]:
x.shape

torch.Size([1, 16, 11, 11])

In [31]:
# next pooling layer
x = F.max_pool2d(x,2,2)

In [32]:
x.shape

torch.Size([1, 16, 5, 5])

In [34]:
# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    # fully connected layer
    self.fc1 = nn.Linear(5*5*16,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

    #forward function
  def forward(self,X):
    X =F.relu(self.conv1(X))
    X =F.max_pool2d(X,2,2)
    # Second pass
    X =F.relu(self.conv2(X))
    X =F.max_pool2d(X,2,2)
    # re-view data to flatten it out
    X = X.view(-1,16*5*5)
    # fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X,dim=1)




In [35]:
# create an instance of our model and set a seed
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [36]:
# loss function optimiser
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(),lr=0.001)
# smaller lr = longer runtime
